In [ ]:
import sys
!{sys.executable} -m pip install scikit-learn


In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor


In [19]:
df = pd.read_csv('/Users/t2023-m0090/Downloads/2025_Airbnb_NYC_listings.csv')  

In [ ]:
# 한글 깨짐 방지 (필요시)
plt.rcParams['font.family'] = 'AppleGothic'  # macOS
# plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows
plt.rcParams['axes.unicode_minus'] = False

# price object형 → float 변환
df['price'] = df['price'].replace('[\$,]', '', regex=True).astype(float)

#  여기부터 전처리 코드

In [ ]:
# 전처리 host_listings_count - 결측치 제거, 로그 변환 후 이상치 제거

import numpy as np

#결측치 제거
df = df.dropna(subset=['host_listings_count'])

#로그 변환
df['log_host_listings'] = np.log1p(df['host_listings_count'])

# 로그 기준으로 IQR 이상치 제거
q1 = df['log_host_listings'].quantile(0.25)
q3 = df['log_host_listings'].quantile(0.75)
iqr = q3 - q1
lower = q1 - 1.5 * iqr
upper = q3 + 1.5 * iqr

# 이상치 제거
df = df[(df['log_host_listings'] >= lower) & (df['log_host_listings'] <= upper)]


In [ ]:
#전처리  host_is_superhost - 결측치 제거, 이진코딩
import numpy as np

df['host_is_superhost'] = df['host_is_superhost'].astype(str).str.strip()
df['host_is_superhost'] = df['host_is_superhost'].replace('nan', np.nan)
df['host_is_superhost'] = df['host_is_superhost'].map({'t': 1, 'f': 0})
df = df.dropna(subset=['host_is_superhost'])


In [ ]:
#전처리  host_identity_verified - 결측치 제거, 이진코딩
import numpy as np

df['host_identity_verified'] = df['host_identity_verified'].astype(str).str.strip()
df['host_identity_verified'] = df['host_identity_verified'].replace('nan', np.nan)
df['host_identity_verified'] = df['host_identity_verified'].map({'t': 1, 'f': 0})
df = df.dropna(subset=['host_identity_verified'])

In [ ]:
#전처리 - host_response_rate, host_acceptance_rate (타입 변환, 결측치 중앙값으로)
df['host_response_rate'] = df['host_response_rate'].str.rstrip('%').astype(float)
df['host_acceptance_rate'] = df['host_acceptance_rate'].str.rstrip('%').astype(float)
df['host_response_rate'].fillna(df['host_response_rate'].median(), inplace=True)
df['host_acceptance_rate'].fillna(df['host_acceptance_rate'].median(), inplace=True)


In [ ]:
# 전처리 - host_response_time 순서형 범주처리, 결측치 최빈값으로
response_time_mapping = {
    'within an hour': 0,
    'within a few hours': 1,
    'within a day': 2,
    'a few days or more': 3
}
df['host_response_time'] = df['host_response_time'].map(response_time_mapping)

# 결측치 -> 최빈값으로 대체
df['host_response_time'].fillna(df['host_response_time'].mode()[0], inplace=True)


# --------------